In [1]:
import glob
import time
import pandas as pd
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import keras
import os

[nltk_data] Downloading package punkt to C:\Users\Madusha
[nltk_data]     Dilshan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Madusha
[nltk_data]     Dilshan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Madusha
[nltk_data]     Dilshan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df=pd.read_csv('sqli_datasets.csv')

In [3]:
df

,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
...,...,...
30914,DELETE FROM door WHERE grow = 'small',0
30915,DELETE FROM tomorrow,0
30916,SELECT wide ( s ) FROM west,0
30917,SELECT * FROM ( SELECT slide FROM breath ),0


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))

data = vectorizer.fit_transform(df['Query'].values.astype('U')).toarray()


In [5]:
data.shape

(30919, 4096)

In [6]:
data.shape=(30919,64,64,1)

In [7]:
X=data
y=df['Label']

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

trainX = X_train.reshape(X_train.shape[0], -1)

testX = X_test.reshape(X_test.shape[0], -1)


In [9]:
X_train.shape

(24735, 64, 64, 1)

In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [11]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(64,64,1)),

    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),

    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),

    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(256, activation='relu'),

    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [12]:
model.compile(

    loss='binary_crossentropy', 

    optimizer='adam', 

    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 256)        0

In [13]:
classifier_nn = model.fit(
    X_train,
    y_train,
    epochs=10,
    verbose=True,
    validation_data=(X_test, y_test),
    batch_size=128
)


Epoch 1/10
194/194 [==============================] - 193s 986ms/step - loss: 0.1980 - accuracy: 0.9233 - val_loss: 0.1235 - val_accuracy: 0.9591
Epoch 2/10
194/194 [==============================] - 188s 971ms/step - loss: 0.1243 - accuracy: 0.9569 - val_loss: 0.1140 - val_accuracy: 0.9604
Epoch 3/10
194/194 [==============================] - 197s 1s/step - loss: 0.1068 - accuracy: 0.9632 - val_loss: 0.1095 - val_accuracy: 0.9623
Epoch 4/10
194/194 [==============================] - 198s 1s/step - loss: 0.1002 - accuracy: 0.9657 - val_loss: 0.1061 - val_accuracy: 0.9639
Epoch 5/10
194/194 [==============================] - 194s 1s/step - loss: 0.0959 - accuracy: 0.9671 - val_loss: 0.1220 - val_accuracy: 0.9631
Epoch 6/10
194/194 [==============================] - 195s 1s/step - loss: 0.1191 - accuracy: 0.9614 - val_loss: 0.1117 - val_accuracy: 0.9643
Epoch 7/10
194/194 [==============================] - 203s 1s/step - loss: 0.0950 - accuracy: 0.9681 - val_loss: 0.1104 - val_accuracy: 

In [14]:
pred=model.predict(X_test)

194/194 [==============================] - 16s 80ms/step


In [15]:
for i in range(len(pred)):
    if pred[i] > 0.5:
        pred[i] = 1
    elif pred[i] <= 0.5:
        pred[i] = 0

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.9634540750323415

In [17]:
for i,j in zip(y_test,pred):
    print(i==j)

[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]


In [18]:
from keras.models import load_model
import pickle

model.save('../API/app/models/model_cnn.h5')

with open('../API/app/models/vectorizer_cnn', 'wb') as fin:
    pickle.dump(vectorizer, fin)


In [19]:
accuracy_score

<function sklearn.metrics._classification.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)>

In [20]:
import keras
from keras.models import load_model
import pickle


mymodel = tf.keras.models.load_model('../API/app/models/model_cnn.h5')
myvectorizer = pickle.load(open("../API/app/models/vectorizer_cnn", 'rb'))

In [21]:
def clean_data(input_val):
    replacements = {
        '\n': '',
        '%20': ' ',
        '=': ' = ',
        '((': ' (( ',
        '))': ' )) ',
        '(': ' ( ',
        ')': ' ) ',
        '1 ': 'numeric',
        ' 1': 'numeric',
        "'1 ": "'numeric ",
        " 1'": " numeric'",
        '1,': 'numeric,',
        ' 2 ': ' numeric ',
        ' 3 ': ' numeric ',
        ' 3--': ' numeric--',
        ' 4 ': ' numeric ',
        ' 5 ': ' numeric ',
        ' 6 ': ' numeric ',
        ' 7 ': ' numeric ',
        ' 8 ': ' numeric ',
        '1234': ' numeric ',
        '22': ' numeric ',
        ' 200 ': ' numeric ',
        '23 ': ' numeric ',
        '"1': '"numeric',
        '1"': '"numeric',
        '7659': 'numeric',
        ' 37 ': ' numeric ',
        ' 45 ': ' numeric '
    }

    for key, value in replacements.items():
        input_val = input_val.replace(key, value)

    return input_val


In [22]:
def predict_sqli_attack(text):

    input_val = clean_data(text)
    
    input_val = [input_val]
    input_val = myvectorizer.transform(input_val).toarray()
    input_val.shape = (1, 64, 64, 1)

    result = mymodel.predict(input_val)

    if result > 0.5: #Then it's seems like Malicious SQL query
        return 1
    elif result <= 0.5: #Then it seems like safe
        return 0


In [23]:
predict_sqli_attack("d")

1/1 [==============================] - 0s 114ms/step


0